In [20]:
import pandas as pd
import plotnine as p
import anndata as ad
import scanpy as sc
import scanpy.external as sce
import numpy as np
import matplotlib as mpl
import bbknn
import re
import os

In [2]:
anno = ''

In [4]:
adata = sc.read('data/01_compartments.h5ad')

In [5]:
adata = adata.raw.to_adata()

In [ ]:
min_cells_subcluster = 100

In [6]:
default_res = 0.8

res = {'Astrocytes': default_res,
 'B': default_res,
 'Blood.Endothelial': default_res,
 'Chondrocyte': default_res,
 'Cumulus': default_res,
 'Dendritic': default_res,
 'Epi.NonSecretory': default_res,
 'Epi.Secretory': default_res,
 'Fibroblast': default_res,
 'Lymph.Endothelial': default_res,
 'MAST': default_res,
 'Melanocyte': default_res,
 'Microglia': default_res,
 'Mono.Mac': default_res,
 'Neuron.GABA': default_res,
 'Neuron.GLU': default_res,
 'OPC': default_res,
 'Oligodendrocyte': default_res,
 'Plasma': default_res,
 'Smooth.Muscle': default_res,
 'T': 1.5}

In [10]:
adata_sub = adata[adata.obs['Broad_Celltype'] == anno, ]

x = adata_sub.obs.value_counts('sample') 
x = x[x > 50].index.astype(str)
adata_sub = adata_sub[adata_sub.obs['sample'].isin(x), :]
sc.pp.highly_variable_genes(adata_sub, n_top_genes=2200, flavor="seurat_v3", batch_key='project', span=0.75)
adata_sub.raw = adata_sub

adata_sub.var.loc[adata_sub.var['mt'], 'highly_variable'] = False
adata_sub.var.loc[adata_sub.var_names.str.match('^IG[HIKL]'), 'highly_variable'] = False
    
sc.pp.normalize_total(adata_sub)
sc.pp.log1p(adata_sub)
sc.tl.pca(adata_sub)

# bbknn.bbknn(adata_sub, batch_key='sample', n_pcs=20)
# sc.tl.umap(adata_sub)
# adata_sub.obs['sub_umap_bbknn_1'] = adata_sub.obsm['X_umap'][:,0]
# adata_sub.obs['sub_umap_bbknn_2'] = adata_sub.obsm['X_umap'][:,1]
# sc.pl.umap(adata_sub, color='project')

sce.pp.harmony_integrate(adata_sub, 'sample')
sc.pp.neighbors(adata_sub, n_pcs=20, use_rep='X_pca_harmony')
sc.tl.leiden(adata_sub, resolution=res[anno])
sc.tl.umap(adata_sub)
sc.tl.tsne(adata_sub, n_pcs= 20, use_rep='X_pca_harmony')

adata_sub.obs['leiden'] = anno +'.'+ adata_sub.obs["leiden"].astype(str)

x = adata_sub.obs.pivot_table(columns=['leiden'],index=['Narrow_Celltype'], fill_value=0, aggfunc='size')
x = x.idxmax(axis=0)[adata_sub.obs.leiden].to_frame().values
adata_sub.obs['leiden_vote'] = x

# x = adata_sub.obs.value_counts('leiden') 
# x = x[x >= min_cells_subcluster ].index.astype(str)
# adata_sub = adata_sub[adata_sub.obs['leiden'].isin(x), :]

adata_sub.obs['sub_umap_harmony_1'] = adata_sub.obsm['X_umap'][:,0]
adata_sub.obs['sub_umap_harmony_2'] = adata_sub.obsm['X_umap'][:,1]
#adata_sub.obs['sub_tsne_harmony_1'] = adata_sub.obsm['X_tsne'][:,0]
#adata_sub.obs['sub_tsne_harmony_2'] = adata_sub.obsm['X_tsne'][:,1]

In [ ]:
adata_sub.write('data/02_subcluster/' +anno + '.h5ad')
adata_sub.obs.to_csv('data/02_subcluster/' +anno+ '.csv', index_label='barcode')

In [ ]:
col = "Narrow_Celltype"

In [ ]:
if len(adata_sub.obs[col].unique()) > 1:
    sc.tl.rank_genes_groups(adata_sub, col, method="wilcoxon", use_raw=False)
    for x in adata_sub.obs[col].unique().categories:
        print(anno + "  " + x)
        y = re.sub(r"[\s\/]", "_", x)
        res = sc.get.rank_genes_groups_df(adata_sub, group=x)
        os.makedirs(f"out/DE/{col}_subtype/", exist_ok=True)
        res.to_csv(f"out/DE/{col}_subtype/{y}.tab.gz", sep="\t", index=False)

In [ ]:
col = "leiden_vote"

In [ ]:
if len(adata_sub.obs[col].unique()) > 1:
    sc.tl.rank_genes_groups(adata_sub, col, method="wilcoxon", use_raw=False)
    for x in adata_sub.obs[col].unique().categories:
        print(anno + "  " + x)
        y = re.sub(r"[\s\/]", "_", x)
        res = sc.get.rank_genes_groups_df(adata_sub, group=x)
        os.makedirs(f"out/DE/{col}_subtype/", exist_ok=True)
        res.to_csv(f"out/DE/{col}_subtype/{y}.tab.gz", sep="\t", index=False)

In [ ]:
col = "leiden"

In [ ]:
if len(adata_sub.obs[col].unique()) > 1:
    sc.tl.rank_genes_groups(adata_sub, col, method="wilcoxon", use_raw=False)
    for x in adata_sub.obs[col].unique().categories:
        print(anno + "  " + x)
        y = re.sub(r"[\s\/]", "_", x)
        res = sc.get.rank_genes_groups_df(adata_sub, group=x)
        os.makedirs(f"out/DE/{col}_subtype/", exist_ok=True)
        res.to_csv(f"out/DE/{col}_subtype/{y}.tab.gz", sep="\t", index=False)